***
## Table of Contents
<br>
  <br>
    <ol>
        <h2><a href='#1' style="text-decoration : none; color : #000000;">1. Initial Models</a></h2>
        <ul>
            <a href='#1.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#1.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br> 

<ol>
        <h2><a href='#2' style="text-decoration : none; color : #000000;">2. Oversampling </a></h2>
        <ul>
            <a href='#2.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#2.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>
     
 <ol>
        <h2><a href='#3' style="text-decoration : none; color : #000000;">3. Undersampling</a></h2>
        <ul>
            <a href='#3.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#3.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>
     
 <ol>
        <h2><a href='#4' style="text-decoration : none; color : #000000;">4.Feature Transformation with feature selection</a></h2>
        <ul>
            <a href='#4.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
        <ul>
            <a href='#4.B' style="text-decoration : none; color : #000000;"><h3>B. Average Feature Importance   </h3></a>
        </ul>
     </ol>
     <br>
     
     
 <ol>
        <h2><a href='#5' style="text-decoration : none; color : #000000;">5.Feature Transformation with PCA</a></h2>
        <ul>
            <a href='#5.A' style="text-decoration : none; color : #000000;"><h3>A. Results </h3></a>
        </ul>
     </ol>
     <br>

In [34]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display,clear_output,HTML
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)

import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
warnings.simplefilter("ignore", DeprecationWarning)
import researchpy as rp

%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',305)

from sklearn.feature_selection import SelectKBest,chi2,RFE,RFECV,f_regression,SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn import metrics



from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,auc,roc_curve,recall_score,precision_score,matthews_corrcoef

from pandas_ml import ConfusionMatrix


from imblearn.over_sampling import SMOTE

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import featuretools as ft

from scipy import stats
import re

from xgboost import XGBClassifier
from xgboost import plot_importance

In [35]:
### Function to plot Feature Importance
def feat_impt_plot(feat_impt,y_labels,width_s=1000,height_s=1000):
    
    m=pd.DataFrame(y_labels,feat_impt).reset_index()
    m.columns=['Feature_Importance','Features']
    m.sort_values(by='Feature_Importance',inplace=True,ascending=False)
    m['Features']=m['Features'].str.replace("dom_function_","")
    m['Features']=m['Features'].str.replace("js_function_","")
    m['Features']=m['Features'].apply(lambda x: str(x).lstrip('.'))
    m['Features']=m['Features'].str.replace("(","")
    
    data = [go.Bar(x=m.Feature_Importance.values,y=m.Features.values,text=np.round(m.Feature_Importance,4),
            textposition = 'outside',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ),
            opacity=0.6,
            orientation='h'
        )]
    layout = go.Layout(autosize=False,
    width=width_s,
    height=height_s,
    xaxis=dict(title='Feature Importances',
        tickfont=dict(
            size=12,
            color='black'
        )),
    yaxis=dict(automargin=True))
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
def uni_value(data):
    lst=[]
    for col in data.columns[:]:
        lst.append([col,dict(data[col].value_counts()),data[col].value_counts().values[0]>=len(data)*0.95])
    col_values=pd.DataFrame(lst,columns=['Feature','Value_Counts','95_%_same_values'])
    col_values.set_index('Feature',drop=True,inplace=True)
    del lst
    return col_values

    
def check_cat(data):
    cat_var=[]
    num_var=[]
    for col in data.columns:
        if col!='Target':
            if len(data[col].value_counts())==2:
                cat_var.append(col)
            else:
                num_var.append(col)
    return cat_var,num_var


def num_stats(data,columns):
    lst=[]
    c=0
    for col in columns:
        corr=stats.pointbiserialr(data[col],data.Target)[0]
        corr_p=stats.pointbiserialr(data[col],data.Target)[1]
        ttest=stats.ttest_ind(data[data.Target==1][col], data[data.Target==0][col], equal_var=False)[0]
        ttest_p=stats.ttest_ind(data[data.Target==1][col], data[data.Target==0][col], equal_var=False)[1]
        lst.append((corr,corr_p,ttest,ttest_p,col))
    stats_data=pd.DataFrame(lst,columns=['Pt. Biserial Corr','Pt. Biserial Corr p-val','ttest','ttest p-val','Feature'])
    stats_data.set_index('Feature',drop=True,inplace=True)
    stats_data.sort_values(by='ttest',inplace=True)
    del lst
    return stats_data


### Generate statistics for categorical data
def cat_stats(data,columns):
    lst=[]
    c=0
    for col in columns:
        if col!='Target':
            table, results = rp.crosstab(data['Target'], data[col], prop= 'col', test= 'chi-square')
            x=results['results'].values.tolist()
            x.append(col)
            lst.append(x)
    chi_df=pd.DataFrame(lst,columns=['Pearson Chi-square','p-value','Cramers V','Feature'])
    chi_df.set_index('Feature',drop=True,inplace=True)
    chi_df.sort_values(by='Cramers V',ascending=False,inplace=True)
    del lst
    return chi_df

def remove_feat(data):
    col_no_unique=[]
    for col in data.columns[:]:
        if len(data[col].value_counts())<=1:
#             print(col)
            col_no_unique.append(col)
    return col_no_unique

### Generate plot for class balance for categorical nature
def class_bal(data,title=''):
    d = [go.Bar(x=data.Target.value_counts().index.values,
               y=data.Target.value_counts().values,
               text=data.Target.value_counts().values,
               textposition = 'outside',
            )]
    layout = go.Layout(title=title,xaxis=dict(title='Domain Type',tickfont=dict(size=12,color='black')),
                    yaxis=dict(title='Number of Instances'))
    fig = go.Figure(data=d, layout=layout)
    iplot(fig)

In [36]:
def model_build(model,X_train,Y_train,X_test,Y_test,tree=False,feat=None):
    
    y_pred_train=model.predict(X_train)
    acc_train=accuracy_score(Y_train,y_pred_train)
    
    print("Accuracy of the model for training data is:",acc_train)
    print("Confusion Matrix for training data is:")
    cm_train=ConfusionMatrix(Y_train,y_pred_train)
    display(cm_train)
    
    
    y_pred_test=model.predict(X_test)
    acc_test=accuracy_score(Y_test,y_pred_test)
    print("Accuracy of the model for test data is:",acc_test)
    print("Confusion Matrix for test data is:")
    cm_test=ConfusionMatrix(Y_test,y_pred_test)
    display(cm_test)

    fpr, tpr, threshold = roc_curve(Y_test, y_pred_test)
    roc_auc =auc(fpr, tpr)
    mcc=matthews_corrcoef(Y_test,y_pred_test)
    if tree==True:
        feat_impt_plot(model.feature_importances_,feat)
        return model,cm_train,cm_test,acc_train,acc_test,roc_auc,model.feature_importances_,mcc
    
    return model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc

def KNN_model(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'K-Nearest Neighbors' + '\033[0m')
    knn=KNeighborsClassifier()
    knn.fit(X_train,Y_train)

    return model_build(knn,X_train,Y_train,X_test,Y_test)

def Logistic_model(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'Logistic Regression' + '\033[0m')
    log=LogisticRegression(random_state=0)
    log.fit(X_train,Y_train)
    
    return model_build(log,X_train,Y_train,X_test,Y_test)

def RandomForest(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'RandomForest Classifier' + '\033[0m')
    rf=RandomForestClassifier(random_state=0)
    rf.fit(X_train,Y_train)

    
    return model_build(rf,X_train,Y_train,X_test,Y_test,True,feat)

def AdaBoost(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'Adaoost Classifier' + '\033[0m')
    ada=AdaBoostClassifier(random_state=0)
    ada.fit(X_train,Y_train)
    
    return model_build(ada,X_train,Y_train,X_test,Y_test,True,feat)

def GradientBoosting(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'GradientBoosting Classifier' + '\033[0m')
    gbc=GradientBoostingClassifier(random_state=0)
    gbc.fit(X_train,Y_train)
    
    return model_build(gbc,X_train,Y_train,X_test,Y_test,True,feat)

def ExtraTree(X_train,Y_train,X_test,Y_test,feat):
    print('\033[1m' + 'ExtraTree Classifier' + '\033[0m')
    ext_tree=ExtraTreesClassifier(random_state=0)
    ext_tree.fit(X_train,Y_train)
    
    return model_build(ext_tree,X_train,Y_train,X_test,Y_test,True,feat)

def Bagging(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'Bagging Classifier' + '\033[0m')
    bagging=BaggingClassifier(random_state=0)
    bagging.fit(X_train,Y_train)
    
    return model_build(bagging,X_train,Y_train,X_test,Y_test)

def DeepLearning1(X_train,Y_train,X_test,Y_test):
    print('\033[1m' + 'Neural Network-1' + '\033[0m')
    np.random.seed(0)
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=X_train.shape[1]))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    deep_learning.fit(X_train,Y_train,batch_size=20,epochs=80,verbose=False)
    acc_train=deep_learning.evaluate(X_train,Y_train,verbose=False)[1]
    print("The accuracy of the model on training data is:",acc_train)
    cm_train=ConfusionMatrix(Y_train,deep_learning.predict_classes(X_train,batch_size=1,verbose=False).reshape(1,len(X_train))[0])
    cm_test=ConfusionMatrix(Y_test,deep_learning.predict_classes(X_test,batch_size=1,verbose=False).reshape(1,len(X_test))[0])
    display(cm_train)
    acc_test=accuracy_score(Y_test,deep_learning.predict_classes(X_test,batch_size=1,verbose=False))
    print("The accuracy of the model on test data is:",acc_test)
    display(cm_test)
    mcc=matthews_corrcoef(Y_test,deep_learning.predict_classes(X_test,batch_size=1,verbose=False))
    fpr, tpr, threshold = roc_curve(Y_test, deep_learning.predict_classes(X_test,batch_size=1,verbose=False))
    roc_auc =auc(fpr, tpr)
    return  deep_learning,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc

### HTML Read Data

In [37]:
df_html=pd.read_csv('../html/HTML.csv',index_col='domain')
df_html.drop(['Unnamed: 0'],axis=1,inplace=True)
cookie=[col for col in df_html.columns if 'number' in col]
df_html.drop(cookie,axis=1,inplace=True)
print("Shape of HTML dataset:",df_html.shape)
df_html.Target=df_html.Target.apply(lambda x: 1 if x=='Malicious' else 0)

count_columns=[col for col in df_html.columns if col.endswith('count')]
df_html['total_count']=df_html[count_columns].sum(axis=1)

href_columns=[col for col in df_html.columns if (('href_absolute' in col) | ('href_relative' in col) | ('href_page' in col))  ]
df_html['total_href']=df_html[href_columns].sum(axis=1)

total_img_src=[col for col in df_html.columns if 'img_src_' in col]
df_html['total_img_src']=df_html[total_img_src].sum(axis=1)

Shape of HTML dataset: (43491, 1461)


In [38]:
html_columns=['a_count','div_count','embed_count','frame_count','iframe_count','img_count','input_count',
              'link_count','meta_count','object_count','style_count','title_count','total_count','total_href',
              'total_img_src','iframe_hidden_true','script_type_text/javascript','a_href_out_of_domain',
 'area_href_out_of_domain',
 'audio_src_out_of_domain',
 'base_href_out_of_domain',
 'embed_src_out_of_domain',
 'form_action_out_of_domain',
 'frame_src_out_of_domain',
 'iframe_src_out_of_domain',
 'img_src_out_of_domain',
 'img_srcset_out_of_domain',
 'link_href_out_of_domain',
 'object_data_out_of_domain',
 'script_src_out_of_domain',
 'source_src_out_of_domain',
 'source_srcset_out_of_domain',
 'video_src_out_of_domain','Target']

df_html=df_html[html_columns]

### JavaScript read data

In [39]:
df_js=pd.read_csv('Javascript.csv',low_memory=False,index_col='domain')

cookie=[col for col in df_js.columns if 'number' in col]
df_js.drop(cookie,axis=1,inplace=True)

df_js.rename(columns={'Malicious':'Target'},inplace=True)

df_js.Target=df_js.Target.apply(lambda x: 1 if x=='Malicious' else 0)

js_columns=[ 'dom_function_document.createElement(',
 'dom_function_document.write(',
 'dom_function_window.addEventListener(',
 'dom_function_window.setInterval(',
 'dom_function_window.setTimeout(',

 'js_function_.charCodeAt(',
 'js_function_.concat(',
 'js_function_.exec(',
 'js_function_.link(',
 'js_function_.search(',
 'js_function_.substring(',


 'js_function_escape(',
 'js_function_eval(',
 'js_function_parseInt(',
 'js_function_String.fromCharCode(',
 'js_function_unescape(',
'js_function_.replace(','Target'
]

In [40]:
df_js=df_js[js_columns]

In [41]:
df=pd.concat([df_js,df_html],axis=1,join='inner')
df['Target_z']=df[['Target','Target']].apply(max,axis=1)
df.drop(['Target','Target'],axis=1,inplace=True)
df.rename(columns={'Target_z':'Target'},inplace=True)

In [44]:
df_sel=df.copy(deep=True)

### <a id='1' style="text-decoration : none; color : #000000;"> 2. Initial Models</a>

In [10]:
train=df_sel.iloc[:28496,:]
train_X=train.iloc[:,train.columns!='Target'].values
train_Y=train.Target.values

scaler=StandardScaler()
scaled_X_train=scaler.fit_transform(train_X)

test=df_sel.iloc[28496:,:]
test_X=test.iloc[:,test.columns!='Target'].values
test_Y=test.Target.values
scaled_X_test=scaler.transform(test_X)

In [45]:
columns=df_sel.columns.values.tolist()
columns.remove('Target')

In [13]:
ind_columns=['KNeighborsClassifier','LogisticRegression','RandomForestClassifier','AdaBoostClassifier',
             'GradientBoostingClassifier','ExtraTreeClassifier','BaggingClassifier','Neural Network-1']

f=[KNN_model,Logistic_model,RandomForest,AdaBoost,GradientBoosting, ExtraTree,Bagging,DeepLearning1]

## Table to store training and test measures
table_80_50_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_50_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','MCC'])

feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y,columns)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y)
    table_80_50_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_50_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9131457046603032
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      22327  2013    24340
True         462  3694     4156
__all__    22789  5707    28496

Accuracy of the model for test data is: 0.90927699991812
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       9543   895    10438
True         213  1562     1775
__all__     9756  2457    12213



Logistic Regression
Accuracy of the model for training data is: 0.8932481751824818
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       9557   881    10438
True         211  1564     1775
__all__     9768  2445    12213



Neural Network-1
The accuracy of the model on training data is: 0.9079519932622122


Predicted  False  True  __all__
Actual                         
False      22208  2132    24340
True         491  3665     4156
__all__    22699  5797    28496

The accuracy of the model on test data is: 0.9006796037009743


Predicted  False  True  __all__
Actual                         
False       9479   959    10438
True         254  1521     1775
__all__     9733  2480    12213

### <a id='1.A' style="text-decoration : none; color : #000000;"> A Results</a>

In [14]:
trace0 = go.Scatter(x=table_80_50_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_50_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_50_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_50_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_50_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_50_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_50_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_50_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_50_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [15]:
table_80_50_train['index']=ind_columns
table_80_50_test['index']=ind_columns


table_80_50_test.set_index(['index'],drop=True,inplace=True)
table_80_50_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_50_train)
print("Test Results")
display(table_80_50_test)

Training Results


,accuracy,TP,TN,FP,FN,FPR,FNR
index,,,,,,,
KNeighborsClassifier,0.913146,3694.0,22327.0,2013.0,462.0,0.082703,0.111165
LogisticRegression,0.893248,3173.0,22281.0,2059.0,983.0,0.084593,0.236526
RandomForestClassifier,0.925428,3951.0,22420.0,1920.0,205.0,0.078882,0.049326
AdaBoostClassifier,0.896512,3145.0,22402.0,1938.0,1011.0,0.079622,0.243263
GradientBoostingClassifier,0.908724,3508.0,22387.0,1953.0,648.0,0.080238,0.155919
ExtraTreeClassifier,0.927850,4016.0,22424.0,1916.0,140.0,0.078718,0.033686
BaggingClassifier,0.925358,3952.0,22417.0,1923.0,204.0,0.079006,0.049086
Neural Network-1,0.907952,3665.0,22208.0,2132.0,491.0,0.087592,0.118142


Test Results


,accuracy,TP,TN,FP,FN,FPR,FNR,auc,MCC
index,,,,,,,,,
KNeighborsClassifier,0.909277,1562.0,9543.0,895.0,213.0,0.085744,0.120000,0.897128,0.698280
LogisticRegression,0.893147,1375.0,9533.0,905.0,400.0,0.086702,0.225352,0.843973,0.622235
RandomForestClassifier,0.911815,1562.0,9574.0,864.0,213.0,0.082774,0.120000,0.898613,0.704237
AdaBoostClassifier,0.895194,1358.0,9575.0,863.0,417.0,0.082679,0.234930,0.841196,0.623507
GradientBoostingClassifier,0.905920,1498.0,9566.0,872.0,277.0,0.083541,0.156056,0.880201,0.677663
ExtraTreeClassifier,0.911733,1567.0,9568.0,870.0,208.0,0.083349,0.117183,0.899734,0.705018
BaggingClassifier,0.910587,1564.0,9557.0,881.0,211.0,0.084403,0.118873,0.898362,0.701735
Neural Network-1,0.900680,1521.0,9479.0,959.0,254.0,0.091876,0.143099,0.882513,0.670247


### <a id='1.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [16]:
feat_impt_plot(np.average(feat_imp,axis=0),columns)

### <a id='2' style="text-decoration : none; color : #000000;"> 2. OverSampling </a>

In [17]:
sm = SMOTE(random_state=12, ratio = 1.0)
X_train, Y_train = sm.fit_sample(scaled_X_train,train_Y)

/nethomes/darshan/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning:

Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.



In [18]:
## Table to store training and test measures so we can compare later
table_80_80_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_80_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](X_train,Y_train,scaled_X_test,test_Y,columns)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](X_train,Y_train,scaled_X_test,test_Y)
    table_80_80_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_80_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.6001027115858669
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      23585   755    24340
True       18712  5628    24340
__all__    42297  6383    48680

Accuracy of the model for test data is: 0.8397609105051994
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       9958   480    10438
True        1477   298     1775
__all__    11435   778    12213



Logistic Regression
Accuracy of the model for training data is: 0.88594905505341
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       9358  1080    10438
True         151  1624     1775
__all__     9509  2704    12213



Neural Network-1
The accuracy of the model on training data is: 0.9301972062448645


Predicted  False   True  __all__
Actual                          
False      21872   2468    24340
True         930  23410    24340
__all__    22802  25878    48680

The accuracy of the model on test data is: 0.8916728076639646


Predicted  False  True  __all__
Actual                         
False       9277  1161    10438
True         162  1613     1775
__all__     9439  2774    12213

### <a id='2.A' style="text-decoration : none; color : #000000;"> A Results</a>

In [19]:
trace0 = go.Scatter(x=table_80_80_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_80_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_80_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_80_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_80_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_80_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_80_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_80_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_80_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [20]:
table_80_80_train['index']=ind_columns
table_80_80_test['index']=ind_columns


table_80_80_test.set_index(['index'],drop=True,inplace=True)
table_80_80_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_80_train)
print("Test Results")
display(table_80_80_test)

Training Results


,accuracy,TP,TN,FP,FN,FPR,FNR
index,,,,,,,
KNeighborsClassifier,0.600103,5628.0,23585.0,755.0,18712.0,0.031019,0.768776
LogisticRegression,0.885949,23232.0,19896.0,4444.0,1108.0,0.182580,0.045522
RandomForestClassifier,0.951479,24203.0,22115.0,2225.0,137.0,0.091413,0.005629
AdaBoostClassifier,0.901027,23144.0,20718.0,3622.0,1196.0,0.148809,0.049137
GradientBoostingClassifier,0.921118,23488.0,21352.0,2988.0,852.0,0.122761,0.035004
ExtraTreeClassifier,0.952301,24238.0,22120.0,2220.0,102.0,0.091208,0.004191
BaggingClassifier,0.951233,24192.0,22114.0,2226.0,148.0,0.091454,0.006081
Neural Network-1,0.930197,23410.0,21872.0,2468.0,930.0,0.101397,0.038209


Test Results


,accuracy,TP,TN,FP,FN,FPR,FNR,auc,mcc
index,,,,,,,,,
KNeighborsClassifier,0.839761,298.0,9958.0,480.0,1477.0,0.045986,0.832113,0.560951,0.175917
LogisticRegression,0.837468,1684.0,8544.0,1894.0,91.0,0.181452,0.051268,0.883640,0.594170
RandomForestClassifier,0.900925,1617.0,9386.0,1052.0,158.0,0.100786,0.089014,0.905100,0.690972
AdaBoostClassifier,0.861787,1650.0,8875.0,1563.0,125.0,0.149741,0.070423,0.889918,0.624216
GradientBoostingClassifier,0.881438,1640.0,9125.0,1313.0,135.0,0.125790,0.076056,0.899077,0.656987
ExtraTreeClassifier,0.900843,1616.0,9386.0,1052.0,159.0,0.100786,0.089577,0.904818,0.690585
BaggingClassifier,0.899206,1624.0,9358.0,1080.0,151.0,0.103468,0.085070,0.905731,0.688819
Neural Network-1,0.891673,1613.0,9277.0,1161.0,162.0,0.111228,0.091268,0.898752,0.670849


### <a id='2.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [21]:
feat_impt_plot(np.average(feat_imp,axis=0),columns)

### <a id='3' style="text-decoration : none; color : #000000;"> 3. Undersampling</a>

In [22]:
df_non=train[train.Target==0]
df_mal=train[train.Target==1]

df_non=df_non.sample(frac=len(df_mal)/len(df_non),random_state=0)
df_u=df_non.append(df_mal)
df_u.Target.value_counts() 

X_train=df_u[columns]
X_train=scaler.transform(X_train)
Y_train=df_u.Target.values

print("Shape after undersampling",X_train.shape)

## Table to store training and test measures so we can compare later
table_80_90_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_90_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](X_train,Y_train,scaled_X_test,test_Y,columns)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](X_train,Y_train,scaled_X_test,test_Y)
    table_80_90_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_90_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

Shape after undersampling (8312, 50)
K-Nearest Neighbors
Accuracy of the model for training data is: 0.5696583253128008
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False       4016   140     4156
True        3437   719     4156
__all__     7453   859     8312

Accuracy of the model for test data is: 0.8400065503971178
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False      10002   436    10438
True        1518   257     1775
__all__    11520   693    12213



Logistic Regression
Accuracy of the model for training data is: 0.8783686236766122
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       9158  1280    10438
True         125  1650     1775
__all__     9283  2930    12213



Neural Network-1
The accuracy of the model on training data is: 0.9169874879692012


Predicted  False  True  __all__
Actual                         
False       3543   613     4156
True          77  4079     4156
__all__     3620  4692     8312

The accuracy of the model on test data is: 0.8507328256775567


Predicted  False  True  __all__
Actual                         
False       8697  1741    10438
True          82  1693     1775
__all__     8779  3434    12213

### <a id='3.A' style="text-decoration : none; color : #000000;"> A. Results</a>

In [23]:
trace0 = go.Scatter(x=table_80_90_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_90_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_90_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_90_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_90_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_90_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_90_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_90_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_90_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [24]:
table_80_90_train['index']=ind_columns
table_80_90_test['index']=ind_columns


table_80_90_test.set_index(['index'],drop=True,inplace=True)
table_80_90_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_90_train)
print("Test Results")
display(table_80_90_test)

Training Results


,accuracy,TP,TN,FP,FN,FPR,FNR
index,,,,,,,
KNeighborsClassifier,0.569658,719.0,4016.0,140.0,3437.0,0.033686,0.826997
LogisticRegression,0.878369,3955.0,3346.0,810.0,201.0,0.194899,0.048364
RandomForestClassifier,0.947425,4096.0,3779.0,377.0,60.0,0.090712,0.014437
AdaBoostClassifier,0.893407,3915.0,3511.0,645.0,241.0,0.155197,0.057988
GradientBoostingClassifier,0.908205,3918.0,3631.0,525.0,238.0,0.126323,0.057267
ExtraTreeClassifier,0.951756,4093.0,3818.0,338.0,63.0,0.081328,0.015159
BaggingClassifier,0.947425,4099.0,3776.0,380.0,57.0,0.091434,0.013715
Neural Network-1,0.916987,4079.0,3543.0,613.0,77.0,0.147498,0.018527


Test Results


,accuracy,TP,TN,FP,FN,FPR,FNR,auc,mcc
index,,,,,,,,,
KNeighborsClassifier,0.840007,257.0,10002.0,436.0,1518.0,0.041770,0.855211,0.551509,0.156938
LogisticRegression,0.828380,1691.0,8426.0,2012.0,84.0,0.192757,0.047324,0.879959,0.582684
RandomForestClassifier,0.890526,1650.0,9226.0,1212.0,125.0,0.116114,0.070423,0.906732,0.676834
AdaBoostClassifier,0.857119,1668.0,8800.0,1638.0,107.0,0.156927,0.060282,0.891396,0.620922
GradientBoostingClassifier,0.873250,1647.0,9018.0,1420.0,128.0,0.136041,0.072113,0.895923,0.643541
ExtraTreeClassifier,0.900761,1628.0,9373.0,1065.0,147.0,0.102031,0.082817,0.907576,0.692963
BaggingClassifier,0.884959,1650.0,9158.0,1280.0,125.0,0.122629,0.070423,0.903474,0.666002
Neural Network-1,0.850733,1693.0,8697.0,1741.0,82.0,0.166794,0.046197,0.893504,0.616971


### <a id='3.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [25]:
feat_impt_plot(np.average(feat_imp,axis=0),columns)

### <a id='4' style="text-decoration : none; color : #000000;">4. Feature Transformation with feature selection </a>

In [64]:
df_sel.reset_index(inplace=True)

# columns=columns.tolist()
# columns.append('domain')

df_min=df_sel[columns]

es = ft.EntitySet(id = 'malicious')
es.entity_from_dataframe(entity_id = 'data', dataframe = df_min, index = 'domain')

feature_matrix_sessions, features_defs = ft.dfs(entityset=es,target_entity="data",
                                                trans_primitives = ['add','divide','multiply'],n_jobs=-1,
                                                verbose=1,max_depth=1)

Built 4950 features
EntitySet scattered to workers in 4.904 seconds
Elapsed: 00:34 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [65]:
df_sel.set_index(['domain'],drop=True,inplace=True)
fs=pd.merge(feature_matrix_sessions,pd.DataFrame(df_sel.Target),left_index=True, right_index=True)
fs.replace([np.inf],0,inplace=True)
fs.fillna(value=0,inplace=True)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1223, in _close
    quiet_exceptions=(CancelledError,),
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 584, in with_timeout
    chain_future(future_converted, result)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 166, in chain_future
    future_add_done_callback(a, copy)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 262, in future_add_done_callback
    callback(future)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 160, in copy
    elif a.ex

In [48]:
feature_name=['total_href / total_count',
 'link_href_out_of_domain / link_count',
 'img_count / total_count',
 'img_count + total_count',
 'div_count + total_count',
 'a_href_out_of_domain / total_href',
 'a_href_out_of_domain / total_count',
 'a_count / total_count',
 'total_img_src / total_count',
 'total_count + total_img_src',
 'title_count + total_count',
 'style_count + total_count',
 'source_srcset_out_of_domain + total_count',
 'script_type_text/javascript + total_count',
 'script_src_out_of_domain + total_count',
 'meta_count + total_count',
 'link_href_out_of_domain / total_href',
 'link_href_out_of_domain + total_count',
 'link_count + total_count',
 'js_function_unescape( + total_count',
 'js_function_parseInt( + total_count',
 'js_function_eval( + total_count',
 'js_function_escape( + total_count',
 'js_function_String.fromCharCode( + total_count',
 'js_function_.substring( + total_count',
 'js_function_.search( + total_count',
 'js_function_.replace( + total_count',
 'js_function_.link( + total_count',
 'js_function_.exec( + total_count',
 'js_function_.concat( + total_count',
 'js_function_.charCodeAt( + total_count',
 'input_count + total_count',
 'img_srcset_out_of_domain + total_count',
 'img_src_out_of_domain / total_img_src',
 'img_src_out_of_domain / total_count',
 'img_src_out_of_domain + total_count',
 'iframe_src_out_of_domain + total_count',
 'iframe_count + total_count',
 'form_action_out_of_domain + total_count',
 'dom_function_window.setTimeout( + total_count',
 'dom_function_window.addEventListener( + total_count',
 'dom_function_document.write( + total_count',
 'dom_function_document.createElement( + total_count',
 'dom_function_document.createElement( + script_src_out_of_domain',
 'div_count / total_count',
 'base_href_out_of_domain + total_count',
 'area_href_out_of_domain + total_count',
 'a_href_out_of_domain / a_count','Target']

In [49]:
print('Features we will be using:\n')
display(feature_name)

Features we will be using:



['total_href / total_count',
 'link_href_out_of_domain / link_count',
 'img_count / total_count',
 'img_count + total_count',
 'div_count + total_count',
 'a_href_out_of_domain / total_href',
 'a_href_out_of_domain / total_count',
 'a_count / total_count',
 'total_img_src / total_count',
 'total_count + total_img_src',
 'title_count + total_count',
 'style_count + total_count',
 'source_srcset_out_of_domain + total_count',
 'script_type_text/javascript + total_count',
 'script_src_out_of_domain + total_count',
 'meta_count + total_count',
 'link_href_out_of_domain / total_href',
 'link_href_out_of_domain + total_count',
 'link_count + total_count',
 'js_function_unescape( + total_count',
 'js_function_parseInt( + total_count',
 'js_function_eval( + total_count',
 'js_function_escape( + total_count',
 'js_function_String.fromCharCode( + total_count',
 'js_function_.substring( + total_count',
 'js_function_.search( + total_count',
 'js_function_.replace( + total_count',
 'js_function_.li

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1223, in _close
    quiet_exceptions=(CancelledError,),
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 584, in with_timeout
    chain_future(future_converted, result)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 166, in chain_future
    future_add_done_callback(a, copy)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 262, in future_add_done_callback
    callback(future)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 160, in copy
    elif a.ex

In [50]:
fs=fs[feature_name]

In [53]:
train=fs.iloc[:28496]

In [54]:
train_Y=train.Target.values

In [56]:
train_X=train.iloc[:,train.columns!="Target"].values

In [57]:
scaler=StandardScaler()
scaled_X_train=scaler.fit_transform(train_X)

test=fs.iloc[28496:]
test_Y=test.Target.values
test_X=test.iloc[:,train.columns!="Target"].values

scaled_X_test=scaler.transform(test_X)

In [58]:
feature_name.remove('Target')

In [59]:
## Table to store training and test measures so we can compare later
table_80_60_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_60_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])


feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y,feature_name)
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](scaled_X_train,train_Y,scaled_X_test,test_Y)
    table_80_60_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_60_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9184797866367209
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      22833  2028    24861
True         295  3340     3635
__all__    23128  5368    28496

Accuracy of the model for test data is: 0.8864324899697044
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       9045   872     9917
True         515  1781     2296
__all__     9560  2653    12213



Logistic Regression
Accuracy of the model for training data is: 0.8837731611454239
Confusion Matrix for training data is:


...

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1223, in _close
    quiet_exceptions=(CancelledError,),
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 584, in with_timeout
    chain_future(future_converted, result)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 166, in chain_future
    future_add_done_callback(a, copy)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 262, in future_add_done_callback
    callback(future)
  File "/nethomes/darshan/anaconda3/lib/python3.6/site-packages/tornado/concurrent.py", line 160, in copy
    elif a.exception() is not None:
concurrent.futures._base.CancelledError
Exception ignored in: <bound method

The accuracy of the model on training data is: 0.8951080853453116


Predicted  False  True  __all__
Actual                         
False      22213  2648    24861
True         341  3294     3635
__all__    22554  5942    28496

The accuracy of the model on test data is: 0.8693195774993859


Predicted  False  True  __all__
Actual                         
False       8828  1089     9917
True         507  1789     2296
__all__     9335  2878    12213

### <a id='4.A' style="text-decoration : none; color : #000000;"> A. Results</a>

In [60]:
trace0 = go.Scatter(x=table_80_60_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_60_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_60_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_60_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_60_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_60_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_60_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_60_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_60_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [61]:
table_80_60_train['index']=ind_columns
table_80_60_test['index']=ind_columns


table_80_60_test.set_index(['index'],drop=True,inplace=True)
table_80_60_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_60_train)
print("Test Results")
display(table_80_60_test)

Training Results


,accuracy,TP,TN,FP,FN,FPR,FNR
index,,,,,,,
KNeighborsClassifier,0.918480,3340.0,22833.0,2028.0,295.0,0.081574,0.081155
LogisticRegression,0.883773,3035.0,22149.0,2712.0,600.0,0.109087,0.165062
RandomForestClassifier,0.926832,3496.0,22915.0,1946.0,139.0,0.078275,0.038239
AdaBoostClassifier,0.911882,3173.0,22812.0,2049.0,462.0,0.082418,0.127098
GradientBoostingClassifier,0.917287,3236.0,22903.0,1958.0,399.0,0.078758,0.109766
ExtraTreeClassifier,0.928657,3545.0,22918.0,1943.0,90.0,0.078155,0.024759
BaggingClassifier,0.926937,3500.0,22914.0,1947.0,135.0,0.078315,0.037139
Neural Network-1,0.895108,3294.0,22213.0,2648.0,341.0,0.106512,0.093810


Test Results


,accuracy,TP,TN,FP,FN,FPR,FNR,auc,mcc
index,,,,,,,,,
KNeighborsClassifier,0.886432,1781.0,9045.0,872.0,515.0,0.087930,0.224303,0.843884,0.651658
LogisticRegression,0.861787,1707.0,8818.0,1099.0,589.0,0.110820,0.256533,0.816324,0.587582
RandomForestClassifier,0.891918,1805.0,9088.0,829.0,491.0,0.083594,0.213850,0.851278,0.667404
AdaBoostClassifier,0.878490,1686.0,9043.0,874.0,610.0,0.088131,0.265679,0.823095,0.620275
GradientBoostingClassifier,0.884467,1712.0,9090.0,827.0,584.0,0.083392,0.254355,0.831126,0.637625
ExtraTreeClassifier,0.892410,1821.0,9078.0,839.0,475.0,0.084602,0.206882,0.854258,0.670680
BaggingClassifier,0.891918,1818.0,9075.0,842.0,478.0,0.084905,0.208188,0.853454,0.669157
Neural Network-1,0.869320,1789.0,8828.0,1089.0,507.0,0.109811,0.220819,0.834685,0.616225


### <a id='4.B' style="text-decoration : none; color : #000000;"> B. Average Feature Importances</a>

In [62]:
feat_impt_plot(np.average(feat_imp,axis=0),feature_name)

### <a id='5' style="text-decoration : none; color : #000000;">5. Feature Transformation with PCA </a>

In [67]:
X=fs.loc[:,fs.columns!='Target']
y=fs.Target.values
feature_name = X.columns.tolist()
X=np.nan_to_num(X)
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

In [68]:
pca=PCA(n_components=300,whiten=True,random_state=0)
X_pca_21=pca.fit_transform(scaled_X)
print("Variance explained using 215 components is:",sum(pca.explained_variance_ratio_))

Variance explained using 215 components is: 0.8119849715062663


In [71]:
X_train=X_pca_21[:28496]
Y_train=fs.iloc[:28496]['Target'].values
X_test=X_pca_21[28496:]
Y_test=fs.iloc[28496:]['Target'].values

## Table to store training and test measures so we can compare later
table_80_70_train=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR'])
table_80_70_test=pd.DataFrame(columns=['accuracy','TP','TN','FP','FN','FPR','FNR','auc','mcc'])

In [72]:
feat_imp=[None]*4
j=0
for i in range(0,8):
    if i>=2 and i<=5:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,feat_imp[j],mcc=f[i](X_train,Y_train,X_test,Y_test,np.arange(0,300).astype(str))
        j=j+1
    else:
        model,cm_train,cm_test,acc_train,acc_test,roc_auc,mcc=f[i](X_train,Y_train,X_test,Y_test)
    table_80_70_train.loc[i]=   ([acc_train,cm_train.TP,cm_train.TN,cm_train.FP,cm_train.FN,cm_train.FPR,cm_train.FNR])
    table_80_70_test.loc[i] =   ([acc_test ,cm_test.TP, cm_test.TN, cm_test.FP, cm_test.FN, cm_test.FPR, cm_test.FNR,roc_auc,mcc])
    print("\n")

K-Nearest Neighbors
Accuracy of the model for training data is: 0.9181990454800674
Confusion Matrix for training data is:


Predicted  False  True  __all__
Actual                         
False      22815  2046    24861
True         285  3350     3635
__all__    23100  5396    28496

Accuracy of the model for test data is: 0.8883976091050519
Confusion Matrix for test data is:


Predicted  False  True  __all__
Actual                         
False       9065   852     9917
True         511  1785     2296
__all__     9576  2637    12213



Logistic Regression
Accuracy of the model for training data is: 0.906478102189781
Confusion Matrix for training data is:


...

Predicted  False  True  __all__
Actual                         
False       9082   835     9917
True         486  1810     2296
__all__     9568  2645    12213



Neural Network-1
The accuracy of the model on training data is: 0.9247262773722628


Predicted  False  True  __all__
Actual                         
False      22840  2021    24861
True         124  3511     3635
__all__    22964  5532    28496

The accuracy of the model on test data is: 0.8880700892491608


Predicted  False  True  __all__
Actual                         
False       9001   916     9917
True         451  1845     2296
__all__     9452  2761    12213

### <a id='5.A' style="text-decoration : none; color : #000000;"> A. Results</a>

In [73]:
trace0 = go.Scatter(x=table_80_70_test.iloc[0].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[0].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'KNN_model')

trace1 = go.Scatter(x=table_80_70_test.iloc[1].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[1].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'Logistic Regression')

trace2 = go.Scatter(x=table_80_70_test.iloc[2].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[2].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Random Forest')


trace3 = go.Scatter(x=table_80_70_test.iloc[3].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[3].values[[0,5,6,7]],
                    mode = 'lines',
                    name = 'Ada Boost')

trace4 = go.Scatter(x=table_80_70_test.iloc[4].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[4].values[[0,5,6,7]],
                    mode = 'lines+markers',
                    name = 'GradientBoosting')

trace5 = go.Scatter(x=table_80_70_test.iloc[5].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[5].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'ExtraTree Classifier')

trace6 = go.Scatter(x=table_80_70_test.iloc[6].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[6].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Bagging')

trace7 = go.Scatter(x=table_80_70_test.iloc[7].index.values[[0,5,6,7]],
                    y=table_80_70_test.iloc[7].values[[0,5,6,7]],
                    mode = 'lines+markers',marker=dict(symbol='x'),
                    name = 'Neural Network')



data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(yaxis=dict(tick0=0,dtick=0.05),title='Result comparision for Features selected in Section 1')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='Result')

In [74]:
table_80_70_train['index']=ind_columns
table_80_70_test['index']=ind_columns


table_80_70_test.set_index(['index'],drop=True,inplace=True)
table_80_70_train.set_index(['index'],drop=True,inplace=True)

print("Training Results")
display(table_80_70_train)
print("Test Results")
display(table_80_70_test)

Training Results


,accuracy,TP,TN,FP,FN,FPR,FNR
index,,,,,,,
KNeighborsClassifier,0.918199,3350.0,22815.0,2046.0,285.0,0.082298,0.078404
LogisticRegression,0.906478,3034.0,22797.0,2064.0,601.0,0.083022,0.165337
RandomForestClassifier,0.927042,3504.0,22913.0,1948.0,131.0,0.078356,0.036039
AdaBoostClassifier,0.914374,3237.0,22819.0,2042.0,398.0,0.082137,0.109491
GradientBoostingClassifier,0.920164,3305.0,22916.0,1945.0,330.0,0.078235,0.090784
ExtraTreeClassifier,0.928727,3547.0,22918.0,1943.0,88.0,0.078155,0.024209
BaggingClassifier,0.927253,3509.0,22914.0,1947.0,126.0,0.078315,0.034663
Neural Network-1,0.924726,3511.0,22840.0,2021.0,124.0,0.081292,0.034113


Test Results


,accuracy,TP,TN,FP,FN,FPR,FNR,auc,mcc
index,,,,,,,,,
KNeighborsClassifier,0.888398,1785.0,9065.0,852.0,511.0,0.085913,0.222561,0.845763,0.656655
LogisticRegression,0.874232,1676.0,9001.0,916.0,620.0,0.092367,0.270035,0.818799,0.609250
RandomForestClassifier,0.892573,1803.0,9098.0,819.0,493.0,0.082585,0.214721,0.851347,0.668642
AdaBoostClassifier,0.880455,1704.0,9049.0,868.0,592.0,0.087526,0.257840,0.827317,0.627304
GradientBoostingClassifier,0.884877,1718.0,9089.0,828.0,578.0,0.083493,0.251742,0.832382,0.639395
ExtraTreeClassifier,0.892901,1810.0,9095.0,822.0,486.0,0.082888,0.211672,0.852720,0.670328
BaggingClassifier,0.891837,1810.0,9082.0,835.0,486.0,0.084199,0.211672,0.852064,0.667890
Neural Network-1,0.888070,1845.0,9001.0,916.0,451.0,0.092367,0.196429,0.855602,0.664318


In [75]:
### Original Creator : Darshan Bhansali
### HTML code to hide the input cells 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle code, click <a href="javascript:code_toggle()">here</a>.''')